Load training and test data from the MNIST dataset

In [7]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Print the shape of the data

In [9]:
print('Train X:', x_train.shape)
print('Train Y:', y_train.shape)
print('Test X:', x_test.shape)
print('Test Y:', y_test.shape)

Train X: (60000, 28, 28)
Train Y: (60000,)
Test X: (10000, 28, 28)
Test Y: (10000,)


Make the inputs compatible with ConvNets

In [10]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1) / 255
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1) / 255

Print the new shape of the data

In [11]:
print('Train X:', x_train.shape)
print('Train Y:', y_train.shape)
print('Test X:', x_test.shape)
print('Test Y:', y_test.shape)

Train X: (60000, 28, 28, 1)
Train Y: (60000,)
Test X: (10000, 28, 28, 1)
Test Y: (10000,)


The following function takes an input and output (target classes) array and generated a number of pairs from them.

In [12]:
import numpy as np
from random import randint
from tqdm import tqdm

def GeneratePairs(x, y, num_pairs):
    assert x.shape[0]==y.shape[0]
    num_samples = x.shape[0]
    num_classes = y.max() + 1 # in the case of mnist, it is 10
    
    #separate x samples based on the y value, this makes it easier to generate an equal number of positive
    # and negative pairs
    separated = [[] for c in range(num_classes)]
    for i in range(num_samples):
        separated[y[i]].append(x[i])
    
    #left input
    l = np.empty(shape=(num_pairs, x.shape[1], x.shape[2], x.shape[3]), dtype=np.float32)
    #right input
    r = np.empty(shape=(num_pairs, x.shape[1], x.shape[2], x.shape[3]), dtype=np.float32)
    #output (class: 0 or 1)
    o = np.empty(shape=(num_pairs, 1), dtype=np.float32)
    
    for i in tqdm(range(num_pairs)):
        if i%2==0: #negative pair
            #randomly pick two differnt classes (y values)
            y_l = randint(0, num_classes-1)
            y_r = (y_l + randint(1, num_classes-2)) % num_classes # make sure y_l != y_r
            #for each class pick a random sample
            x_l = randint(0, len(separated[y_l])-1)
            x_r = randint(0, len(separated[y_r])-1)
            #add the random pair to the data arrays
            l[i, :, :, :] = separated[y_l][x_l][:, :, :]
            r[i, :, :, :] = separated[y_r][x_r][:, :, :]
            o[i, 0] = 0
        else: #positive pair
            #randomly pick a class (y value)
            c = randint(0, num_classes-1)
            #pick two random samples
            num_x = len(separated[c])
            x_l = randint(0, num_x-1)
            x_r = (x_l + randint(1, num_x-2)) % num_x # make sure x_l != x_r
            #add the random pair to the data arrays
            l[i, :, :, :] = separated[c][x_l][:, :, :]
            r[i, :, :, :] = separated[c][x_r][:, :, :]
            o[i, 0] = 1
    return (l, r, o)

Use the previous function to generate training as test pairs

In [13]:
(train_l, train_r, train_y) = GeneratePairs(x_train, y_train, 600000)
(test_l, test_r, test_y) = GeneratePairs(x_test, y_test, 100000)

100%|███████████████████████████████████████████████████████████████████████| 100000/100000 [00:02<00:00, 34960.79it/s]


The following will create the Siamese ConvNet Model

In [14]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Lambda
import keras.backend as K

in_shape = (28, 28, 1)

#Feature Extraction ConvNet
ConvNet = Sequential(name='Feature_CNN')
ConvNet.add(Conv2D(16, kernel_size=3, activation='relu', padding='same', input_shape=in_shape))
ConvNet.add(MaxPooling2D())
ConvNet.add(Conv2D(32, kernel_size=3, activation='relu', padding='same'))
ConvNet.add(MaxPooling2D())
ConvNet.add(Conv2D(64, kernel_size=3, activation='relu', padding='same'))
ConvNet.add(Flatten())

#Siamese ConvNet Creation using Keras Functional API
#Left Input
input_l = Input(shape=in_shape)
#Right Input
input_r = Input(shape=in_shape)
#Left Feature-Vector
encoded_l = ConvNet(input_l)
#Right Feature-Vector
encoded_r = ConvNet(input_r)
#L1 Lambda Layer
#  Merge function
def L1_Merge(x):
    return K.abs(x[0]-x[1])
#  Shape function
def L1_Shape(s):
    return s[0]
#  Combine the two feature-vectors using a Lambda Layer
merged = Lambda(function=L1_Merge, output_shape=L1_Shape)([encoded_l, encoded_r])
#Classification
prediction = Dense(1, activation='sigmoid')(merged)
#Create The Siamese ConvVet model
SiameseNet = Model(name='Siamese_CNN', input=[input_l, input_r], output=prediction)

c:\users\abdal\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:37: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="Siamese_CNN", inputs=[<tf.Tenso..., outputs=Tensor("de...)`


<b>Feature Extraction ConvNet</b>

In [15]:
ConvNet.summary()

Model: "Feature_CNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
Total params: 23,296
Trainable params: 23,296
Non-trainable params: 0
___________________________________________________

<b>Siamese Convnet</b>

In [16]:
SiameseNet.summary()

Model: "Siamese_CNN"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
Feature_CNN (Sequential)        (None, 3136)         23296       input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 3136)         0           Feature_CNN[1][0]      

<b>Compile the model</b>

In [17]:
from keras.optimizers import SGD
SiameseNet.compile(optimizer=SGD(0.05), loss='binary_crossentropy', metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


<b>Train the model</b>

In [18]:
SiameseNet.fit([train_l, train_r], train_y, batch_size=500, epochs=100, verbose=2)


Epoch 1/100
 - 59s - loss: 0.4401 - acc: 0.7842
Epoch 2/100
 - 31s - loss: 0.1802 - acc: 0.9305
Epoch 3/100
 - 31s - loss: 0.1250 - acc: 0.9533
Epoch 4/100
 - 31s - loss: 0.1026 - acc: 0.9620
Epoch 5/100
 - 32s - loss: 0.0888 - acc: 0.9673
Epoch 6/100
 - 33s - loss: 0.0798 - acc: 0.9707
Epoch 7/100
 - 33s - loss: 0.0726 - acc: 0.9733
Epoch 8/100
 - 36s - loss: 0.0665 - acc: 0.9760
Epoch 9/100
 - 36s - loss: 0.0620 - acc: 0.9775
Epoch 10/100
 - 39s - loss: 0.0577 - acc: 0.9792
Epoch 11/100
 - 40s - loss: 0.0542 - acc: 0.9805
Epoch 12/100
 - 40s - loss: 0.0510 - acc: 0.9816
Epoch 13/100
 - 45s - loss: 0.0485 - acc: 0.9826
Epoch 14/100
 - 46s - loss: 0.0461 - acc: 0.9835
Epoch 15/100
 - 49s - loss: 0.0439 - acc: 0.9843
Epoch 16/100
 - 48s - loss: 0.0419 - acc: 0.9850
Epoch 17/100
 - 48s - loss: 0.0400 - acc: 0.9859
Epoch 18/100
 - 48s - loss: 0.0384 - acc: 0.9863
Epoch 19/100
 - 52s - loss: 0.0368 - acc: 0.9870
Epoch 20/100
 - 52s - loss: 0.0355 - acc: 0.9876
Epoch 21/100
 - 50s - loss: 

<b>Evaluate the trained model</b>

In [19]:
SiameseNet.evaluate([test_l, test_r], test_y, verbose=2)

[0.08496359100975695, 0.98333]